# Product Recommendation
Reference: https://ieeexplore.ieee.org/document/5430993

In [19]:
import numpy as np
import msgpack
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

In [2]:
tf.__version__

'2.1.0'

## Data Preprocessing

In [3]:
Y_data = pd.read_csv('data/Y.csv', header=None, names=['Rating','Movie','User'], dtype=int) # training data
P_data = pd.read_csv('data/P.csv', header=None, names=['Rating','Movie','User'], dtype=int) # test data ('probe-set' mentioned in paper)

In [36]:
display(Y_data.head())
display(P_data.head())

,Rating,Movie,User
0,5,2,1
1,4,7,1
2,4,8,1
3,4,11,1
4,4,12,1


,Rating,Movie,User
0,3,6,1
1,5,96,1
2,3,1,2
3,3,33,3
4,5,42,4


In [37]:
Y_data.shape, P_data.shape

((3399874, 3), (189699, 3))

In [87]:
print(Y_data['Rating'].unique().max(), Y_data['Movie'].unique().max(), Y_data['User'].unique().max())
print(P_data['Rating'].unique().max(), P_data['Movie'].unique().max(), P_data['User'].unique().max())

5 100 137328
5 100 137328


In [214]:
k, n = Y_data['Movie'].unique().max(), Y_data['User'].unique().max()
k, n

(100, 137328)

In [215]:
indices = np.reshape(Y_data[['Movie', 'User']].values-1, (-1, 2))
print(indices.shape)
indices

(3399874, 2)


array([[     1,      0],
       [     6,      0],
       [     7,      0],
       ...,
       [    97, 137327],
       [    98, 137327],
       [    99, 137327]])

In [11]:
Z_sparse = tf.SparseTensor(indices=indices, values=Y_data['Rating'].values, dense_shape=[k, n])
Z_sparse = tf.cast(Y_sparse, tf.float32)

In [89]:
%%time
Z_0 = tf.sparse.slice(Z_sparse, [0, 0], [100,1])

CPU times: user 18.7 ms, sys: 839 µs, total: 19.5 ms
Wall time: 18.3 ms


In [12]:
# use dense matrices for faster linear transformations since all matrices can fit in memory
Z = tf.sparse.to_dense(Z_sparse, validate_indices=False)
Z

<tf.Tensor: shape=(100, 137328), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 3.],
       [0., 0., 0., ..., 3., 0., 0.],
       ...,
       [5., 0., 0., ..., 4., 0., 4.],
       [4., 0., 3., ..., 0., 0., 4.],
       [3., 4., 0., ..., 4., 5., 4.]], dtype=float32)>

In [22]:
# memoization
t_Z_dict = {}
t_y_dict = {}
t_x_dict = {}
t_Hy_dict = {}
t_Hx_dict = {}
t_Hy_trans_dict = {}
t_Hx_trans_dict = {}
t_movie_ids_labels_dict = {}
t_labels_dict = {}

for t in tqdm(range(n)):
    movie_ids = Y_data['Movie'][Y_data['User']==t+1].values
    H_yt = tf.constant(np.identity(k)[movie_ids-1], dtype=tf.float32)
    H_xt = tf.constant(np.delete(np.identity(k), movie_ids-1, 0), dtype=tf.float32)
    Z_t = tf.expand_dims(Z[:, t], axis=1)
    y_t = tf.matmul(H_yt, Z_t)
    x_t = tf.matmul(H_xt, Z_t)
    
    # store the variables for fast future reference
    t_Hy_dict[t] = H_yt
    t_Hx_dict[t] = H_xt
    t_Hx_trans_dict[t] = tf.transpose(H_xt)
    t_Hy_trans_dict[t] = tf.transpose(H_yt)
    
    t_x_dict[t] = x_t
    t_y_dict[t] = y_t
    t_Z_dict[t] = Z_t
    
    t_movie_ids_labels_dict[t] = P_data['Movie'][P_data['User']==t+1].values
    t_labels_dict[t] = tf.expand_dims(P_data['Rating'][P_data['User']==t+1].values, axis=1)

100%|██████████| 137328/137328 [13:51<00:00, 165.19it/s] 


## Initialization
$\mu$ has 1 type available <br />
R has 4 types available

In [38]:
# initial estimate of mu
N = 0
H_yty_t = 0
for t in tqdm(range(n)):
    H_yt = t_Hy_dict[t]
    N += tf.matmul(tf.transpose(H_yt), H_yt)
    Z_t = t_Z_dict[t]
    y_t = t_y_dict[t]
    H_yty_t += tf.matmul(tf.transpose(H_yt), y_t)

100%|██████████| 137328/137328 [00:26<00:00, 5134.64it/s]


In [66]:
print(N.shape)
print(H_yty_t.shape)
mu_hat0 = tf.matmul(tf.linalg.inv(N), H_yty_t)
mu_hat0.shape

(100, 100)
(100, 1)


TensorShape([100, 1])

In [75]:
# initial estimates of R (4 types available)
R_hat0_1 = tf.constant(np.identity(k), dtype=tf.float32)
R_hat0_1

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [53]:
S = 0
for t in tqdm(range(n)):
    Hyt = t_Hy_dict[t]
    yt = t_y_dict[t]
    Hytmu_hat_0 = tf.matmul(Hyt, mu_hat_0)
    S += tf.matmul(tf.transpose(Hyt), tf.matmul(yt - Hytmu_hat_0, tf.matmul(tf.transpose(yt - Hytmu_hat_0), Hyt)))

100%|██████████| 137328/137328 [00:35<00:00, 3911.98it/s]


In [77]:
# diag_S is the diagonal matrix consisting of the diagonal elements of S
diag_S = tf.linalg.diag(tf.linalg.tensor_diag_part(S))
R_hat0_2 = tf.matmul(tf.linalg.inv(N), diag_S)
R_hat0_2

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[1.724364  , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.94215506, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.4365153 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.1832098 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.0349715 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.2620207 ]], dtype=float32)>

In [82]:
# R_hat0_3 is not a good initializer when rating variances are far from one
R_hat0_3 = tf.matmul(tf.linalg.sqrtm(tf.linalg.inv(diag_S)), tf.matmul(S, tf.linalg.sqrtm(tf.linalg.inv(diag_S))))
R_hat0_3

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[ 1.        ,  0.07418475, -0.01158332, ..., -0.01462824,
        -0.02215266, -0.01845053],
       [ 0.07418474,  1.        ,  0.03674483, ...,  0.02561698,
         0.03563043,  0.03926768],
       [-0.01158332,  0.03674483,  0.9999999 , ...,  0.10954399,
         0.12823027,  0.1556249 ],
       ...,
       [-0.01462824,  0.02561698,  0.10954399, ...,  1.        ,
         0.19780062,  0.15164362],
       [-0.02215266,  0.03563043,  0.12823027, ...,  0.19780062,
         1.0000001 ,  0.18997027],
       [-0.01845053,  0.03926769,  0.1556249 , ...,  0.15164363,
         0.18997027,  1.        ]], dtype=float32)>

In [83]:
R_hat0_4 = tf.matmul(tf.linalg.sqrtm(tf.linalg.inv(N)), tf.matmul(S, tf.linalg.sqrtm(tf.linalg.inv(N))))
R_hat0_4

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[ 1.724364  ,  0.09455624, -0.01823068, ..., -0.02089477,
        -0.02959407, -0.02721801],
       [ 0.09455624,  0.94215494,  0.04274768, ...,  0.02704705,
         0.03518409,  0.04281832],
       [-0.01823068,  0.04274768,  1.4365153 , ...,  0.14281526,
         0.15635432,  0.20954023],
       ...,
       [-0.02089477,  0.02704705,  0.14281526, ...,  1.1832099 ,
         0.21888836,  0.18530548],
       [-0.02959406,  0.03518409,  0.15635432, ...,  0.21888837,
         1.0349715 ,  0.2171116 ],
       [-0.02721802,  0.04281832,  0.20954023, ...,  0.18530548,
         0.2171116 ,  1.2620206 ]], dtype=float32)>

In [252]:
%%time
a = t_Hx_dict[0]

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.91 µs


In [263]:
%%time
tf.transpose(a)

CPU times: user 149 µs, sys: 23 µs, total: 172 µs
Wall time: 159 µs


<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 2, 3, 4, 5, 5, 5])>

## Expectation Maximization Algorithm

In [237]:
def expectation_maximization(mu, R):
    Hyt_trans_Ryt_inv_Hyt_sum = 0
    Hyt_trans_Ryt_inv_yt_sum = 0
    R_hat_sum = 0
    log_p_hat = 0

    for t in tqdm(range(n)):
        y_t = t_y_dict[t]
        H_xt = t_Hx_dict[t]
        H_xt_trans = t_Hx_trans_dict[t]
        H_yt = t_Hy_dict[t]
        H_yt_trans = t_Hy_trans_dict[t]
        
        # for R estimation
        R_xt = tf.matmul(H_xt, tf.matmul(R, H_xt_trans))
        R_yt = tf.matmul(H_yt, tf.matmul(R, H_yt_trans))
        R_yt_det = tf.linalg.det(R_yt)
        R_yt_inv = tf.linalg.inv(R_yt)
        R_xtyt = tf.matmul(H_xt, tf.matmul(R, H_yt_trans))
        
        mu_yt = tf.matmul(H_yt, mu)
        mu_xt = tf.matmul(H_xt, mu)

        X_t_hat = tf.matmul(R_xtyt, tf.matmul(R_yt_inv, y_t - mu_yt)) + mu_xt
        Z_t_hat = tf.matmul(H_yt_trans, y_t) + tf.matmul(H_xt_trans, X_t_hat)
        R_hat_sum += tf.matmul(Z_t_hat - mu, tf.transpose(Z_t_hat - mu)) + \
                    tf.matmul(H_xt_trans, 
                              tf.matmul(R_xt - tf.matmul(R_xtyt, tf.matmul(R_yt_inv, tf.transpose(R_xtyt))), 
                                        H_xt)
                             )

        # for mu estimation
        Hyt_trans_Ryt_inv_Hyt_sum += tf.matmul(H_yt_trans, tf.matmul(R_yt_inv, H_yt))
        Hyt_trans_Ryt_inv_yt_sum += tf.matmul(H_yt_trans, tf.matmul(R_yt_inv, y_t))
        
        log_p_hat += -1/2*(tf.math.log(R_yt_det) + tf.matmul(tf.transpose(y_t - mu_yt), tf.matmul(R_yt_inv, y_t - mu_yt))\
                            + k*tf.math.log(2*np.pi))
    
    R_hat = R_hat_sum / n
    mu_hat = tf.matmul(tf.linalg.inv(Hyt_trans_Ryt_inv_Hyt_sum), Hyt_trans_Ryt_inv_yt_sum)    
    return mu_hat, R_hat, log_p_hat

In [155]:
delta = 0.0005
mu = mu_hat0
R = R_hat0_4
log_p = -np.inf

for i in range(30):
    if i % 5 == 0:
        print(f'iteration: {i}')
    
    mu_hat, R_hat, log_p_hat = expectation_maximization(mu, R)
    
    print('normalized log_p_hat:', log_p_hat/n)
    print('normalized log_p:', log_p/n)
    print('diff:', log_p_hat/n - log_p/n)
    if log_p_hat/n - log_p/n < delta:
        break
        
    # use new estimattions for next iteration
    mu = mu_hat
    R = R_hat
    log_p = log_p_hat

  0%|          | 130/137328 [00:00<03:41, 618.72it/s]

iteration: 0


  0%|          | 69/137328 [00:00<03:20, 684.93it/s]

normalized log_p_next: tf.Tensor([[-101.37641]], shape=(1, 1), dtype=float32)
normalized log_p: -inf
diff: tf.Tensor([[inf]], shape=(1, 1), dtype=float32)


  0%|          | 71/137328 [00:00<03:16, 699.02it/s]

...

normalized log_p_next: tf.Tensor([[-100.560814]], shape=(1, 1), dtype=float32)
normalized log_p: tf.Tensor([[-100.56209]], shape=(1, 1), dtype=float32)
diff: tf.Tensor([[0.00127411]], shape=(1, 1), dtype=float32)


  0%|          | 72/137328 [00:00<03:12, 713.93it/s]

normalized log_p_next: tf.Tensor([[-100.55973]], shape=(1, 1), dtype=float32)
normalized log_p: tf.Tensor([[-100.560814]], shape=(1, 1), dtype=float32)
diff: tf.Tensor([[0.00108337]], shape=(1, 1), dtype=float32)


100%|██████████| 137328/137328 [03:09<00:00, 725.78it/s]

normalized log_p_next: tf.Tensor([[-100.559326]], shape=(1, 1), dtype=float32)
normalized log_p: tf.Tensor([[-100.55973]], shape=(1, 1), dtype=float32)
diff: tf.Tensor([[0.00040436]], shape=(1, 1), dtype=float32)


In [181]:
# 22 iterations, 66 min
np.save('results/em_mu.npy', mu_hat)
np.save('results/em_R.npy', R_hat)
np.save('results/em_log_p.npy', log_p_hat)

## McMichael’s Algorithm

In [260]:
def mcmichael(mu, R):
    gamma = 0.00001
    Hyt_trans_Ryt_inv_Hyt_sum = 0
    Hyt_trans_Ryt_inv_yt_sum = 0
    log_p_gradient = 0
    log_p_hat = 0

    for t in tqdm(range(n)):
        y_t = t_y_dict[t]
        H_yt = t_Hy_dict[t]
        H_yt_trans = t_Hy_trans_dict[t]
        
        # for R estimation
        R_yt = tf.matmul(H_yt, tf.matmul(R, H_yt_trans))
        R_yt_inv = tf.linalg.inv(R_yt)
        mu_yt = tf.matmul(H_yt, mu)
        
        log_p_gradient += tf.matmul(H_yt_trans, 
                                    tf.matmul(R_yt_inv - tf.matmul(R_yt_inv, 
                                                                   tf.matmul(y_t - mu_yt, 
                                                                             tf.matmul(tf.transpose(y_t - mu_yt), R_yt_inv))
                                                                  ), H_yt
                                             )
                                   )

        # for mu estimation
        Hyt_trans_Ryt_inv_Hyt_sum += tf.matmul(H_yt_trans, tf.matmul(R_yt_inv, H_yt))
        Hyt_trans_Ryt_inv_yt_sum += tf.matmul(H_yt_trans, tf.matmul(R_yt_inv, y_t))
        
        log_p_hat += -1/2*(tf.math.log(R_yt_det) + tf.matmul(tf.transpose(y_t - mu_yt), tf.matmul(R_yt_inv, y_t - mu_yt))\
                            + k*tf.math.log(2*np.pi))


    R_hat = R + gamma*tf.matmul(R, tf.matmul(-1/2*log_p_gradient, R))
    mu_hat = tf.matmul(tf.linalg.inv(Hyt_trans_Ryt_inv_Hyt_sum), Hyt_trans_Ryt_inv_yt_sum)
    return mu_hat, R_hat, log_p_hat

In [261]:
delta = 0.0005
mu = mu_hat0
R = R_hat0_4
log_p = -np.inf

for i in range(30):
    if i % 5 == 0:
        print(f'iteration: {i}')
    
    mu_hat, R_hat, log_p_hat = mcmichael(mu, R)
    
    print('normalized log_p_hat:', log_p_hat/n)
    print('normalized log_p:', log_p/n)
    print('diff:', log_p_hat/n - log_p/n)
    if log_p_hat/n - log_p/n < delta:
        break
        
    # use new estimattions for next iteration
    mu = mu_hat
    R = R_hat
    log_p = log_p_hat

  0%|          | 112/137328 [00:00<02:03, 1111.70it/s]

iteration: 0


  0%|          | 135/137328 [00:00<01:41, 1346.19it/s]

normalized log_p_hat: tf.Tensor([[-99.37206]], shape=(1, 1), dtype=float32)
normalized log_p: -inf
diff: tf.Tensor([[inf]], shape=(1, 1), dtype=float32)


100%|██████████| 137328/137328 [01:37<00:00, 1401.79it/s]


normalized log_p_hat: tf.Tensor([[-99.371666]], shape=(1, 1), dtype=float32)
normalized log_p: tf.Tensor([[-99.37206]], shape=(1, 1), dtype=float32)
diff: tf.Tensor([[0.00039673]], shape=(1, 1), dtype=float32)


In [262]:
# 2 iterations, 3 min
np.save('results/mcmichael_mu.npy', mu_hat)
np.save('results/mcmichael_R.npy', R_hat)
np.save('results/mcmichael_log_p.npy', log_p_hat)

## Evaluation

In [273]:
def evaluate(mu, R):
    square_error = 0
    l = 0
    for t in tqdm(range(n)):
        movie_ids_t = t_movie_ids_labels_dict[t]
        labels_t = t_labels_dict[t]

        # calculate X_t_hat
        y_t = t_y_dict[t]
        H_xt = t_Hx_dict[t]
        H_xt_trans = t_Hx_trans_dict[t]
        H_yt = t_Hy_dict[t]
        H_yt_trans = t_Hy_trans_dict[t]

        R_xt = tf.matmul(H_xt, tf.matmul(R, H_xt_trans))
        R_yt = tf.matmul(H_yt, tf.matmul(R, H_yt_trans))
        R_yt_inv = tf.linalg.inv(R_yt)

        R_xtyt = tf.matmul(H_xt, tf.matmul(R, H_yt_trans))
        mu_yt = tf.matmul(H_yt, mu)
        mu_xt = tf.matmul(H_xt, mu)

        X_t_hat = tf.matmul(R_xtyt, tf.matmul(R_yt_inv, y_t - mu_yt)) + mu_xt

        # clip ratings
        predictions_t = tf.matmul(H_xt_trans, X_t_hat).numpy()[movie_ids_t-1]
        predictions_t[predictions_t > 5] = 5
        predictions_t[predictions_t < 1] = 1
        
        # accumulate square_error and l
        square_error += tf.matmul(tf.transpose(labels_t - predictions_t), labels_t - predictions_t)
        l += len(labels_t)
    return np.sqrt(numerator/l)

In [259]:
em_mu = np.load('results/em_mu.npy')
em_R = np.load('results/em_R.npy')
rmse = evaluate(em_mu, em_R)
rmse

100%|██████████| 137328/137328 [01:22<00:00, 1659.57it/s]


array([[0.85423294]])

In [274]:
mcmichael_mu = np.load('results/mcmichael_mu.npy')
mcmichael_R = np.load('results/mcmichael_R.npy')
rmse = evaluate(mcmichael_mu, mcmichael_R)
rmse

100%|██████████| 137328/137328 [01:21<00:00, 1674.96it/s]


array([[0.85423294]])